In [ ]:
from DatabenchDB import DatabenchDB, DataMode
from MultiAgentsText2SQL import State, QueryOutput, MultiAgentsText2SQL, MultiAgentTypeMode
from MultiAgentsText2SQL_Tester import MultiAgentsText2SQL_Tester
from Util import SaveList2File, CreateDirectory, CreateAnalysisReport
from configparser import ConfigParser, ExtendedInterpolation
import httpx
from langchain_openai import AzureChatOpenAI
from databench_eval import Runner, Evaluator
from databench_eval.utils import load_qa
from databench_eval.utils import load_table
from datetime import datetime
import pandas as pd

In [ ]:
#Carrega as configurações da aplicação
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config-v1.x.ini', 'UTF-8')

http_client = httpx.Client(verify='petrobras-ca-root.pem')

#Instancia o LLM, que será usado pelo workflow LangGraph
llm = AzureChatOpenAI(
    openai_api_base=config['OPENAI']['AZURE_OPENAI_BASE_URL'],
    model_name=config['OPENAI']['CHATGPT_MODEL'],
    openai_api_version=config['OPENAI']['OPENAI_API_VERSION'],
    openai_api_key=config['OPENAI']['OPENAI_API_KEY'],
    http_client=http_client,
    temperature=0
)

In [ ]:
#Define os paths 
pathDatasets= "/projetos/hmro/Mestrado/Semeval2025_Task8/datasets/"
pathDatabases= "/projetos/hmro/Mestrado/Semeval2025_Task8/databases/"
pathResults = "/projetos/hmro/Mestrado/Semeval2025_Task8/results/"

In [ ]:
#Carrega a base Question Answer de testes
qa_test = load_qa(name="semeval", split="test")

df_qa_test = qa_test.to_pandas()

datasets = df_qa_test['dataset'].unique()

datasets_ordenados = sorted(datasets)

print(datasets_ordenados)  


In [ ]:
datasets_ordenados = ['076_NBA', '077_Gestational', '078_Fires', '079_Coffee', '080_Books']

# Define o tipo de estrutura de agentes que será executada (MultiAgentTypeMode.PLAN_AND_EXECUTE ou SIMPLE)
multiAgentType = MultiAgentTypeMode.PLAN_AND_EXECUTE

#Executa os testes com SAMPLE e ALL para todas as questões para cada dataset
executions_mode = [DataMode.SAMPLE,DataMode.ALL]

#datasets_ordenados =['066_IBM_HR','067_TripAdvisor']

#Relatório consolidade com as metricas das inferencias por dataset e mode (ALL ou SAMPLE)
df_consolidated = pd.DataFrame(columns=['dataset', 'datamode', 'numQuestions', 'acc'])

execution_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

#Criar subdiretorio para os resultados deste teste
testDirPath =f"{pathResults}{execution_timestamp}"
CreateDirectory(testDirPath)

for execution_mode in executions_mode:
    
    #Executa os testes para cada DataSet
    for dataset_name in datasets_ordenados:

        #Carrega o database
        databenchDB = DatabenchDB(pathDatasets, pathDatabases, dataset_name, execution_mode)

        #Carrega o grafo de agentes
        multiAgentsText2SQL= MultiAgentsText2SQL(llm, multiAgentType, databenchDB)

        #Carrega o testador
        multiAgentsText2SQL_Tester = MultiAgentsText2SQL_Tester(multiAgentsText2SQL,execution_mode)
    
        #filtra a base teste Question and Answer pelo dataset
        qa_test_filter = qa_test.filter(lambda x: x['dataset'] == dataset_name)


        #Salva o dataframe com QA desta base
        df_qa_test_filter = qa_test_filter.to_pandas()
        df_qa_test_filter.to_csv(f"{testDirPath}/qa_{dataset_name}_{execution_mode.name}_{execution_timestamp}.csv")
                
        #Executa a inferência para todas as questões da base corrente      
        responses = Runner(multiAgentsText2SQL_Tester.call, 
                                  qa=qa_test_filter, 
                                  prompt_generator=multiAgentsText2SQL_Tester.prompt_generator                                 
                                 ).run( save=f"{testDirPath}/model_responses_{dataset_name}_{execution_mode.name}_{execution_timestamp}.txt")

        #Avalia a acurácia
        evaluator = Evaluator(qa=qa_test_filter)

        #Retorna a acurácia
        acc = evaluator.eval(responses, lite=True if (execution_mode == DataMode.SAMPLE) else False)

        #Salva arquivo com o resultado das avaliações
        SaveList2File(evaluator.evals,f"{testDirPath}/eval_{dataset_name}_{execution_mode.name}_{execution_timestamp}.txt")
              
        # Criar um DataFrame temporário com a linha que deseja adicionar
        nova_linha = pd.DataFrame([{'dataset': dataset_name, 'datamode': execution_mode.name,'numQuestions': qa_test_filter.num_rows,'acc': acc}])
        
        # Concatenar os DataFrames
        df_consolidated = pd.concat([df_consolidated, nova_linha], ignore_index=True)      
                       
        
# Salvar como arquivo CSV o relatorio consolidado com a acuracia de todas as bases de teste
df_consolidated.to_csv(f"{testDirPath}/relatorio_avaliacao_{execution_timestamp}.csv", index=False)       

# Salvar como arquivo CSV o consolidado de todas as questões, respostas de referencia, comparadas com as 
# respostas do modelo e avaliação para as bases SAMPLE e FULL
CreateAnalysisReport(testDirPath)

In [ ]:
#Carrega o database
dataset_name = '066_IBM_HR'
execution_mode= DataMode.SAMPLE
databenchDB = DatabenchDB(pathDatasets, pathDatabases, dataset_name, execution_mode)

#Carrega o grafo de agentes
multiAgentsText2SQL= MultiAgentsText2SQL(llm, MultiAgentTypeMode.PLAN_AND_EXECUTE, databenchDB)

multiAgentsText2SQL.Invoke("Are there more employees who travel frequently than those who work in the HR department?")           
